In [1]:
!pip install transformers



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
print(os.getcwd())


c:\Users\Admin\OneDrive - Hanoi University of Science and Technology\Desktop\MusicChatbot\model\mood_prediction


In [3]:
import sys
import os

# Thêm thư mục chứa mood_data.py và mood_model.py vào sys.path
sys.path.append('/content/MusicChatbot/model/mood_prediction')

# Kiểm tra path hiện tại (không bắt buộc)
print(os.getcwd())


c:\Users\Admin\OneDrive - Hanoi University of Science and Technology\Desktop\MusicChatbot\model\mood_prediction


In [6]:
from mood_data import BuildData
from mood_model import BuildModel
import pandas as pd
from sklearn.model_selection import train_test_split

ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
from mood_data import BuildData

# Khởi tạo với mô hình bạn đang dùng (distilbert)
data_builder = BuildData(csv_path="/content/MusicChatbot/data/processed/mood_prediction_model/filtered_dataset.csv", model_name="distilbert-base-uncased")

# Load và chuẩn hóa dữ liệu
dataset = data_builder.load_and_prepare_data()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/338 [00:00<?, ? examples/s]

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

# Đọc file CSV để in ra bảng gốc (nếu bạn muốn nhìn rõ hơn)
df = pd.read_csv("/content/MusicChatbot/data/processed/mood_prediction_model/filtered_dataset.csv")[['lyrics', 'mood']].dropna()
print(df.head(5))


                                              lyrics     mood
0  friends told her she was better off at the bot...  sadness
1  well i heard it playing soft from a drunken ba...  sadness
2  oh my god did i just say that out loud should'...      joy
3  remember when i called you on the telephone yo...      joy
4  we've known working in obscurity and we'll kno...  sadness


In [ ]:
# In vài mẫu token hóa
print(dataset['train'][0])  # hoặc dùng [i] với các index khác


{'label': tensor(2), 'input_ids': tensor([  101,  2066,  5055,  2006,  1037,  2152,  2240,  2027,  2240,  2039,
         2012,  2305,  2051,  2012,  1996,  3347,  2027,  2035,  2320,  2020,
         2293, 12887,  2085,  2630, 12887,  2024,  2035,  2008,  2027,  2024,
         2027,  5565,  1999,  3109,  1996,  3371,  2027,  3062,  2013,  2293,
         1005,  1055,  3712,  1998,  2085,  2027,  3246,  1999,  1996,  4511,
         2008,  2027,  1005,  2222,  2424,  1037,  2047,  2126,  2000,  4875,
         1037,  2047,  2126,  2000,  4875,  2521,  2185,  2013,  9119,  2682,
         1996,  8044,  1998,  1996,  4542,  1996,  5758,  1998,  3255,  1998,
         1996,  4000,  2008,  2027,  5390,  2085,  1996, 10800,  1005,  1055,
         2042,  4342,  2027,  1005,  2310,  2035,  8007,  1998,  5296,  2021,
         2027,  2064,  2681,  2009,  2369,  2065,  2027,  2071,  2074,  2424,
         1037,  2047,  2126,  2000,  4875,  2011,  1996,  2203,  1997,  1996,
         2305,  2027,  1005,  

In [ ]:
from mood_model import BuildModel  # Giả sử bạn đã lưu class huấn luyện vào model.py

model_builder = BuildModel()
trainer = model_builder.get_trainer(dataset)
trainer.train()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [5]:
import transformers
print(transformers.__version__)

4.54.0
